# Block (re-)construction from header/transactions.

In [1]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include","/usr/local/Cellar/zeromq/4.2.5/include")
#pragma cling add_library_path("/usr/local/lib","/usr/local/Cellar/zeromq/4.2.5/lib")
#pragma cling load("bitcoin","bitcoin-protocol","zmq","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Required header files.
#include <iostream>
#include <string>
#include <bitcoin/protocol.hpp> // Libbitcoin-Protocol

## Fetch header, transaction hashes & transaction data
* **Important:** Server interface is not designed to serve blocks. 
    * Do not query higher than blockheight of 10'000 (too many fetch tx requests)
    * For educational purposes only.
    * If direct block/blockchain access required, consider Libbitcoin `libbitcoin::node::full_node` class
    
* **See [`libbitcoin::chain::block`](https://github.com/libbitcoin/libbitcoin/blob/master/include/bitcoin/bitcoin/chain/block.hpp#L47)** for methods and details related to the block class.    


In [14]:
{
    // Setup of zmq context & socket.
    uint32_t request_counter(0);

    bc::protocol::zmq::context my_context(true); //started
    bc::protocol::zmq::socket dealer_socket(
        my_context,
        bc::protocol::zmq::socket::role::dealer
    );
    bc::code ec;
    bc::config::endpoint public_endpoint("tcp://mainnet1.libbitcoin.net:9091");

    // Connect errors not handled in this example.
    ec = dealer_socket.connect(public_endpoint);

    //--------------------------------------------------------------------------
    // 1) Request block header.
    //--------------------------------------------------------------------------

    bc::protocol::zmq::message header_request;
    bc::chain::header header;

    std::string header_request_command = "blockchain.fetch_block_header";
    uint32_t height(100000);

    header_request.enqueue(bc::to_chunk(header_request_command));
    header_request.enqueue(bc::to_chunk(bc::to_little_endian(request_counter)));
    header_request.enqueue_little_endian(height);

    if ((ec = header_request.send(dealer_socket)))
        {
        std::cout << ec.message() << std::endl;
        return 1;
        }

    // Poll our socket.
    bc::protocol::zmq::poller my_poller;
    my_poller.add(dealer_socket); // socket_id added / multiple sockets possible
    auto dealer_socket_id = dealer_socket.id();
    auto socket_ids = my_poller.wait(2000);

    if (socket_ids.contains(dealer_socket_id))
    {

        bc::protocol::zmq::message server_response;
        server_response.receive(dealer_socket);

        std::string my_message_header_request_command =
            server_response.dequeue_text();

        uint32_t my_message_id;
        server_response.dequeue(my_message_id);

        bc::data_chunk reply_payload;
        server_response.dequeue(reply_payload);

        // Use stream to format response data chunk.
        bc::data_source reply_byte_stream(reply_payload);
        bc::istream_reader reply_byte_stream_reader(reply_byte_stream);

        // Use istream class error read method.
        if ((ec = reply_byte_stream_reader.read_error_code())) {
            // if response begins with error, return.
            std::cout << ec.message() << std::endl;
            return 1;
        }

        header.from_data(bc::to_chunk(reply_byte_stream_reader.read_hash()));
    }


    else
    {
      std::cout << "Request timed out..." << std::endl;
    }

    //--------------------------------------------------------------------------
    // 2) Request tx hashes of block height.
    //--------------------------------------------------------------------------

    std::vector<bc::hash_digest> tx_hashes;

    bc::protocol::zmq::message tx_hashes_request;
    std::string tx_hashes_command = "blockchain.fetch_block_transaction_hashes";

    tx_hashes_request.enqueue(bc::to_chunk(tx_hashes_command));
    tx_hashes_request.enqueue(bc::to_chunk(bc::to_little_endian(request_counter++)));
    tx_hashes_request.enqueue_little_endian(height);

    if ((ec = tx_hashes_request.send(dealer_socket)))
    {
        std::cout << ec.message() << std::endl;
        return 1;
    }

    // Poll our socket.
    socket_ids = my_poller.wait(2000);
    if (socket_ids.contains(dealer_socket_id))
    {
        bc::protocol::zmq::message server_response;
        uint32_t my_message_id;
        bc::data_chunk reply_payload;

        server_response.receive(dealer_socket);

        std::string my_message_header_request_command =
            server_response.dequeue_text();
        server_response.dequeue(my_message_id);
        server_response.dequeue(reply_payload);

        bc::data_source reply_byte_stream(reply_payload);
        bc::istream_reader reply_byte_stream_reader(reply_byte_stream);

        if ((ec = reply_byte_stream_reader.read_error_code())) {
            // if response begins with error, return.
            std::cout << ec.message() << std::endl;
            return 1;
        }

        while (!reply_byte_stream_reader.is_exhausted())
        {
            tx_hashes.push_back(reply_byte_stream_reader.read_hash());
        }
    }

    else
    {
        std::cout << "Request timed out..." << std::endl;
    }

    //--------------------------------------------------------------------------
    // 3) Fetch individual transactions.                               
    //--------------------------------------------------------------------------

    bc::chain::transaction::list tx_list;

    for (const auto tx_hash: tx_hashes)
    {
        // Request transaction.
        bc::protocol::zmq::message transaction_request;
        std::string tx_hashes_command = "blockchain.fetch_transaction2";

        transaction_request.enqueue(bc::to_chunk(tx_hashes_command));
        transaction_request.enqueue(
            bc::to_chunk(bc::to_little_endian(request_counter++)));
        transaction_request.enqueue(bc::to_chunk(tx_hash));

        if ((ec = transaction_request.send(dealer_socket)))
        {
            std::cout << ec.message() << std::endl;
            return 1;
        }

        // Poll our socket.
        socket_ids = my_poller.wait(2000);
        if (socket_ids.contains(dealer_socket_id))
        {
            bc::protocol::zmq::message server_response;
            uint32_t my_message_id;
            bc::data_chunk reply_payload;

            server_response.receive(dealer_socket);

            std::string my_message_header_request_command =
                server_response.dequeue_text();
            server_response.dequeue(my_message_id);
            server_response.dequeue(reply_payload);

            bc::data_source reply_byte_stream(reply_payload);
            bc::istream_reader reply_byte_stream_reader(reply_byte_stream);

            if ((ec = reply_byte_stream_reader.read_error_code())) {
                // if response begins with error, return.
                std::cout << ec.message() << std::endl;
                return 1;
            }

            while (!reply_byte_stream_reader.is_exhausted())
            {
                bc::chain::transaction transaction;
                transaction.from_data(reply_byte_stream_reader, true, true);
                tx_list.push_back(transaction);
            }
        }

        else
        {
            std::cout << "Request timed out..." << std::endl;
        }

    }

    //--------------------------------------------------------------------------
    // 4) Assemble block.
    //--------------------------------------------------------------------------

    bc::chain::block block(header, tx_list);
    auto merkle_root = block.generate_merkle_root();
    std::reverse(std::begin(merkle_root), std::end(merkle_root));
    std::cout << bc::encode_base16(merkle_root)
              << std::endl;

} // End of jupyter cell scope.

f3e94742aca4b5ef85488dc37c06c3282295ffec960994b2c0d5ac2a25a95766


In [11]:
!bx fetch-header --height 100000 -f json | jq ".header.merkle_tree_hash"

"f3e94742aca4b5ef85488dc37c06c3282295ffec960994b2c0d5ac2a25a95766"
